# Libraries

In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from functools import partial
from collections import Counter
import seaborn as sns

from joblib import load

# modeling
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# ignore warnings
import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Get dataset

In [2]:
# get pickle file
providers = load('./data/Providers_Final_2iteration.pkl')
providers.set_index('Provider', inplace=True)

In [3]:
providers

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerAttPhys,Ratio_ClaimsPerPatient,PatientsPerAttPhys,PatientsPerOperPhys,PatientsPerOthPhys,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,Perc_HasTop5AdmitCode,Perc_ClaimsPerTopFraudState,Mean_StatePerAttPhys,Mean_StatePerOperPhys,Mean_StatePerOthPhys,IP_Count_UniquePatients,IP_Perc_MultHosp,IP_Perc_Duplicates,IP_Mean_Duplicate_per_AttPhy,IP_Mean_Duplicate_per_Patient,IP_Perc_Dup_Diff_Provider,IP_Perc_Dup_Diff_State,IP_Count_UniqueState,IP_Mean_PatientsPerAttPhys,IP_Mean_PatientsPerOperPhys,IP_Mean_PatientsPerOtherPhys,IP_Perc_HasAttPhys,IP_Perc_HasNoPhys,IP_Perc_AttPhysIsOperPhys,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_ClaimCostPerAttPhys,IP_Mean_ClaimCostPerOperPhys,IP_Mean_ClaimCostPerOtherPhys,IP_Mean_ClaimCostPerPatient,IP_Perc_No_ProcCode,IP_Sum_DeductibleAmtPaid,IP_Mean_AnnualDeductibleAmt,IP_Sum_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualReimbursementAmt,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_AdmitDurationPerAttPhys,IP_Mean_AdmitDurationPerPatient,IP_Mean_AgeAtClaim,IP_AgeRange,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceThree,IP_Perc_RaceTwo,IP_Perc_HasRenalDisease,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,OP_Count_UniquePatients,OP_Perc_MultHosp,OP_Perc_Duplicates,OP_Mean_Duplicate_per_AttPhy,OP_Mean_Duplicate_per_Patient,OP_Perc_Dup_Diff_Provider,OP_Perc_Dup_Diff_State,OP_Count_UniqueState,OP_Mean_PatientsPerAttPhys,OP_Mean_PatientsPerOperPhys,OP_Mean_PatientsPerOtherPhys,OP_Perc_HasNoPhys,OP_Perc_AttPhysIsOperPhys,OP_Perc_HasAttPhys,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_ClaimCostPerAttPhys,OP_Mean_ClaimCostPerOperPhys,OP_Mean_ClaimCostPerOtherPhys,OP_Mean_ClaimCostPerPatient,OP_Perc_No_DiagCode,OP_Sum_DeductibleAmtPaid,OP_Mean_AnnualDeductibleAmt,OP_Sum_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualReimbursementAmt,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimDuration,OP_Mean_AgeAtClaim,OP_AgeRange,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceThree,OP_Perc_RaceTwo,OP_Perc_HasRenalDisease,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,0,0.800000,1,0.400000,1.785714,1.041667,10,19,16,0.240000,0.040000,0.000000,0.040000,0.0,1.000000,1.000000,1.000000,5.0,0.160000,0.000000,0.0,0.0,0.000000,0.000000,1.0,1.25,1.0,1.0,1.0,0.0,0.000000,20468.000000,4077.360000,19693.000000,24068.000000,43068.0,20468.000000,0.600000,5340.0,897.120000,97000.0,0.878279,17606.000000,12.000000,12.000000,6.000000,6.000000,20468.000000,20468.000000,77.600000,26.0,0.000000,0.400000,1.000000,0.0,0.000000,0.400000,6.000000,0.400000,0.200000,0.800000,0.800000,0.800000,0.800000,0.800000,0.400000,0.000000,0.600000,0.400000,19.0,0.920000,0.550000,1.833333,1.000000,0.550000,0.437500,1.0,2.000000,1.000000,1.285714,0.000000,0.000000,1.000000,382.000000,307.000000,322.950000,333.333333,217.857143,373.421053,0.000000,0.0,463.920000,7640.0,1.000000,2615.200000,12.000000,12.000000,1.550000,77.950000,40.0,0.000000,0.350000,0.800000,0.000000,0.200000,0.300000,5.450000,0.650000,0.200000,0.250000,0.850000,0.750000,0.950000,0.650000,0.400000,0.300000,0.250000,0.200000
P

# Train test split

In [4]:
# we want to normalize and down/upsample after splitting.
scores = providers.drop(['PotentialFraud'], axis=1)
scores = pd.DataFrame(scores, columns = scores.columns)
decision = providers["PotentialFraud"]

Xtrain, Xtest, ytrain, ytest = train_test_split(scores, decision, 
                                                test_size = .30, random_state=0, 
                                                stratify = decision)

In [7]:
providers_trainTestSplit_train = pd.concat([ytrain, Xtrain], axis=1)
display(providers_trainTestSplit_train)
display(providers_trainTestSplit_train['PotentialFraud'].value_counts())

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerAttPhys,Ratio_ClaimsPerPatient,PatientsPerAttPhys,PatientsPerOperPhys,PatientsPerOthPhys,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,Perc_HasTop5AdmitCode,Perc_ClaimsPerTopFraudState,Mean_StatePerAttPhys,Mean_StatePerOperPhys,Mean_StatePerOthPhys,IP_Count_UniquePatients,IP_Perc_MultHosp,IP_Perc_Duplicates,IP_Mean_Duplicate_per_AttPhy,IP_Mean_Duplicate_per_Patient,IP_Perc_Dup_Diff_Provider,IP_Perc_Dup_Diff_State,IP_Count_UniqueState,IP_Mean_PatientsPerAttPhys,IP_Mean_PatientsPerOperPhys,IP_Mean_PatientsPerOtherPhys,IP_Perc_HasAttPhys,IP_Perc_HasNoPhys,IP_Perc_AttPhysIsOperPhys,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_ClaimCostPerAttPhys,IP_Mean_ClaimCostPerOperPhys,IP_Mean_ClaimCostPerOtherPhys,IP_Mean_ClaimCostPerPatient,IP_Perc_No_ProcCode,IP_Sum_DeductibleAmtPaid,IP_Mean_AnnualDeductibleAmt,IP_Sum_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualReimbursementAmt,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_AdmitDurationPerAttPhys,IP_Mean_AdmitDurationPerPatient,IP_Mean_AgeAtClaim,IP_AgeRange,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceThree,IP_Perc_RaceTwo,IP_Perc_HasRenalDisease,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,OP_Count_UniquePatients,OP_Perc_MultHosp,OP_Perc_Duplicates,OP_Mean_Duplicate_per_AttPhy,OP_Mean_Duplicate_per_Patient,OP_Perc_Dup_Diff_Provider,OP_Perc_Dup_Diff_State,OP_Count_UniqueState,OP_Mean_PatientsPerAttPhys,OP_Mean_PatientsPerOperPhys,OP_Mean_PatientsPerOtherPhys,OP_Perc_HasNoPhys,OP_Perc_AttPhysIsOperPhys,OP_Perc_HasAttPhys,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_ClaimCostPerAttPhys,OP_Mean_ClaimCostPerOperPhys,OP_Mean_ClaimCostPerOtherPhys,OP_Mean_ClaimCostPerPatient,OP_Perc_No_DiagCode,OP_Sum_DeductibleAmtPaid,OP_Mean_AnnualDeductibleAmt,OP_Sum_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualReimbursementAmt,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimDuration,OP_Mean_AgeAtClaim,OP_AgeRange,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceThree,OP_Perc_RaceTwo,OP_Perc_HasRenalDisease,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV52041,1,1.000000,0,0.245623,235.625000,2.053377,910,911,916,0.115119,0.000000,0.000000,0.021751,0.037666,7.875000,4.714286,9.500000,0.0,0.041910,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,389.901326,0.0,0.000000,3762.005305,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,918.0,0.846154,0.430960,99.375000,1.542471,0.377214,0.190184,17.0,203.125000,51.000000,288.000000,0.003714,0.134218,0.996286,323.856764,242.538986,321.161526,486.867541,338.796715,303.621408,0.016446,5490.0,653.856764,604980.0,0.984813,2449.395225,11.980371,11.970822,2.445093,74.118302,74.0,0.004775,0.410610,0.666667,0.142857,0.000000,0.198939,4.338462,0.363395,0.138462,0.406897,0.663660,0.592573,0.766578,0.372414,0.328912,0.327321,0.283820,0.094430
PRV51771,0,1.000

0    3433
1     354
Name: PotentialFraud, dtype: int64

In [8]:
providers_trainTestSplit_test = pd.concat([ytest, Xtest], axis=1)
display(providers_trainTestSplit_test)
display(providers_trainTestSplit_test['PotentialFraud'].value_counts())

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerAttPhys,Ratio_ClaimsPerPatient,PatientsPerAttPhys,PatientsPerOperPhys,PatientsPerOthPhys,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,Perc_HasTop5AdmitCode,Perc_ClaimsPerTopFraudState,Mean_StatePerAttPhys,Mean_StatePerOperPhys,Mean_StatePerOthPhys,IP_Count_UniquePatients,IP_Perc_MultHosp,IP_Perc_Duplicates,IP_Mean_Duplicate_per_AttPhy,IP_Mean_Duplicate_per_Patient,IP_Perc_Dup_Diff_Provider,IP_Perc_Dup_Diff_State,IP_Count_UniqueState,IP_Mean_PatientsPerAttPhys,IP_Mean_PatientsPerOperPhys,IP_Mean_PatientsPerOtherPhys,IP_Perc_HasAttPhys,IP_Perc_HasNoPhys,IP_Perc_AttPhysIsOperPhys,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_ClaimCostPerAttPhys,IP_Mean_ClaimCostPerOperPhys,IP_Mean_ClaimCostPerOtherPhys,IP_Mean_ClaimCostPerPatient,IP_Perc_No_ProcCode,IP_Sum_DeductibleAmtPaid,IP_Mean_AnnualDeductibleAmt,IP_Sum_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualReimbursementAmt,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_AdmitDurationPerAttPhys,IP_Mean_AdmitDurationPerPatient,IP_Mean_AgeAtClaim,IP_AgeRange,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceThree,IP_Perc_RaceTwo,IP_Perc_HasRenalDisease,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,OP_Count_UniquePatients,OP_Perc_MultHosp,OP_Perc_Duplicates,OP_Mean_Duplicate_per_AttPhy,OP_Mean_Duplicate_per_Patient,OP_Perc_Dup_Diff_Provider,OP_Perc_Dup_Diff_State,OP_Count_UniqueState,OP_Mean_PatientsPerAttPhys,OP_Mean_PatientsPerOperPhys,OP_Mean_PatientsPerOtherPhys,OP_Perc_HasNoPhys,OP_Perc_AttPhysIsOperPhys,OP_Perc_HasAttPhys,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_ClaimCostPerAttPhys,OP_Mean_ClaimCostPerOperPhys,OP_Mean_ClaimCostPerOtherPhys,OP_Mean_ClaimCostPerPatient,OP_Perc_No_DiagCode,OP_Sum_DeductibleAmtPaid,OP_Mean_AnnualDeductibleAmt,OP_Sum_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualReimbursementAmt,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimDuration,OP_Mean_AgeAtClaim,OP_AgeRange,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceThree,OP_Perc_RaceTwo,OP_Perc_HasRenalDisease,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV53036,0,1.000000,0,0.237705,20.333333,1.768116,63,65,63,0.000000,0.000000,0.000000,0.008197,0.000000,1.000000,1.000000,1.000000,0.0,0.032787,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,577.213115,0.0,0.000000,6735.655738,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.0,0.918033,0.369748,7.333333,1.157895,0.369748,0.257426,1.0,16.666667,6.250000,6.666667,0.000000,0.098361,1.000000,325.491803,241.704565,325.690359,433.995726,242.013889,311.077295,0.024590,180.0,521.885246,39530.0,0.991171,1834.180328,12.000000,11.704918,2.565574,73.803279,73.0,0.008197,0.532787,0.750000,0.000000,0.250000,0.278689,4.803279,0.475410,0.049180,0.516393,0.721311,0.622951,0.827869,0.508197,0.237705,0.409836,0.368852,0.065574
PRV53351,0,1.000000,0,0.388889,18.000000,1.3846

0    1471
1     152
Name: PotentialFraud, dtype: int64

# Scaling

In [9]:
mm_scaler = preprocessing.MinMaxScaler()
Xtrain = mm_scaler.fit_transform(Xtrain)
Xtest = mm_scaler.transform(Xtest)

# Upsampling

In [11]:
### UPSAMPLING
# Separate majority and minority classes
df_majority = providers_trainTestSplit_train[providers_trainTestSplit_train.PotentialFraud==0]
df_minority = providers_trainTestSplit_train[providers_trainTestSplit_train.PotentialFraud==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                # to match majority class
                                 n_samples=providers_trainTestSplit_train['PotentialFraud'].value_counts()[0],
                                 random_state=0) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
display(df_upsampled.PotentialFraud.value_counts())

# separate into Xtrain and Ytrain
Xtrain_upsampled = df_upsampled.drop(['PotentialFraud'], axis=1)
ytrain_upsampled = df_upsampled['PotentialFraud']

1    3433
0    3433
Name: PotentialFraud, dtype: int64

# Modeling

In [12]:
# some functions that will be used in modeling
def plot_feature_importances(model):
    n_features = scores.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), scores_cols.to_numpy()) 
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    

# Stratified Cross Validation needs to be applied 
skf = StratifiedKFold(n_splits = 3, random_state = 0, shuffle = True)

### 1. Logistic regression

In [13]:
logreg = LogisticRegression(random_state=0, class_weight='balanced', penalty='l1', solver='liblinear')

param_grid = {'C': np.logspace(-3,1,100)}

logregCV = GridSearchCV(logreg, 
                        param_grid = param_grid, 
                        scoring='recall',
                        cv=skf).fit(Xtrain,ytrain)

print("Best parameters: {}".format(logregCV.best_params_))
print("Best cross-validation score: {:.2f}".format(logregCV.best_score_))
print("Best estimator:\n{}".format(logregCV.best_estimator_))

Best parameters: {'C': 0.15199110829529347}
Best cross-validation score: 0.87
Best estimator:
LogisticRegression(C=0.15199110829529347, class_weight='balanced', penalty='l1',
                   random_state=0, solver='liblinear')


In [14]:
display(recall_score(ytrain, logregCV.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, logregCV.best_estimator_.predict(Xtest)))

0.8983050847457628

0.9144736842105263

In [70]:
print(confusion_matrix(ytest,logregCV.predict(Xtest)))

[[1242  229]
 [  13  139]]


### 2. Random Forest

In [16]:
forest = RandomForestClassifier(class_weight='balanced', random_state=0) 
forest.fit(Xtrain, ytrain)

display(recall_score(ytrain, forest.predict(Xtrain)))
display(recall_score(ytest, forest.predict(Xtest)))

0.9971751412429378

0.3881578947368421

In [19]:
print('Parameters currently in use:\n')
print(forest.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [57]:
rf_param_grid = {
    'ccp_alpha': [0,1,2],
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10), 
    'max_features': ['auto', 'log2'], 
    'min_samples_leaf': [1, 2, 4], 
    'min_samples_split': [2, 5, 10], 
    'n_estimators': [100, 200, 300, 1000]
}

rf_grid_search = RandomizedSearchCV(RandomForestClassifier(random_state=0, 
                                                  class_weight='balanced'), 
                                    param_distributions=rf_param_grid, cv=skf, scoring='recall', 
                                    random_state=0, 
                                    verbose=2, return_train_score=True)
rf_grid_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(rf_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(rf_grid_search.best_score_))
print("Best estimator:\n{}".format(rf_grid_search.best_estimator_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2, total=   0.3s
[CV] n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2, total=   0.2s
[CV] n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2 
[CV]  n_estimators=100, min_samples_split=10, min_samples_leaf=4, max_features=log2, max_depth=2, criterion=gini, ccp_alpha=2, total=   0.3s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=1, criterion=gini, ccp_alpha=2 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=1, criterion=gini, ccp_alpha=2, total=   2.1s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=1, criterion=gini, ccp_alpha=2 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=1, criterion=gini, ccp_alpha=2, total=   2.4s
[CV] n_estimators=1000, min_samples_split=2, min_samples_l

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.2min finished


Best parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 5, 'criterion': 'gini', 'ccp_alpha': 1}
Best cross-validation score: 1.00
Best estimator:
RandomForestClassifier(ccp_alpha=1, class_weight='balanced', max_depth=5,
                       max_features='log2', min_samples_leaf=4,
                       min_samples_split=10, n_estimators=200, random_state=0)


In [58]:
display(recall_score(ytrain, rf_grid_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, rf_grid_search.best_estimator_.predict(Xtest)))

1.0

1.0

In [61]:
# rf_grid_search.cv_results_['mean_test_score']
rf_grid_search.cv_results_['mean_train_score']

array([0.66666667, 0.33333333, 1.        , 0.66666667, 0.9180791 ,
       0.8700565 , 0.33333333, 0.33333333, 0.96327684, 0.33333333])

In [65]:
rf_param_grid = {
    'max_depth': range(1, 10), 
    'max_features': ['auto', 'log2'], 
    'min_samples_leaf': [4, 5], 
    'min_samples_split': [10, 11], 
    'n_estimators': [150, 200, 250]
}

rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=0, 
                                                  class_weight='balanced'), 
                                    param_grid=rf_param_grid, cv=skf, scoring='recall',
                                    verbose=2, return_train_score=True)
rf_grid_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(rf_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(rf_grid_search.best_score_))
print("Best estimator:\n{}".format(rf_grid_search.best_estimator_))

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150, total=   0.4s
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150, total=   0.3s
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150 
[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=150, total=   0.3s
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200 
[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200, total=   0.4s
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200 
[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200, total=   0.5s
[CV] max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200 
[CV]  max_depth=1, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200, total=   0.4s
[CV] max_depth=1, max_features=auto, min_samples_lea

...

[CV]  max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=200, total=   0.8s
[CV] max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=250 
[CV]  max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=250, total=   0.9s
[CV] max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=250 
[CV]  max_depth=8, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=250, total=   0.9s
[CV] max_depth=8, max_features=log2, min_samples_lea

[CV]  max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=250, total=   1.3s
[CV] max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=250 
[CV]  max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=10, n_estimators=250, total=   1.5s
[CV] max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150 
[CV]  max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150, total=   0.9s
[CV] max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150 
[CV]  max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150, total=   1.0s
[CV] max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150 
[CV]  max_depth=9, max_features=auto, min_samples_leaf=5, min_samples_split=11, n_estimators=150, total=   1.0s
[CV] max_depth=9, max_features=auto, min_samples_lea

[CV]  max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=150, total=   0.7s
[CV] max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200 
[CV]  max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200, total=   0.8s
[CV] max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200 
[CV]  max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200, total=   0.8s
[CV] max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200 
[CV]  max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=200, total=   0.8s
[CV] max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=250 
[CV]  max_depth=9, max_features=log2, min_samples_leaf=5, min_samples_split=11, n_estimators=250, total=   0.9s
[CV] max_depth=9, max_features=log2, min_samples_lea

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed:  8.6min finished


Best parameters: {'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 150}
Best cross-validation score: 0.87
Best estimator:
RandomForestClassifier(class_weight='balanced', max_depth=2, min_samples_leaf=4,
                       min_samples_split=10, n_estimators=150, random_state=0)


In [66]:
display(recall_score(ytrain, rf_grid_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, rf_grid_search.best_estimator_.predict(Xtest)))

0.8926553672316384

0.8881578947368421

### 3. Support vector machine

In [20]:
svm_param_grid = {'C': np.logspace(-3,1,100)}

svm_rand_search = RandomizedSearchCV(SVC(random_state=0, class_weight='balanced'),
                                     svm_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
svm_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(svm_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(svm_rand_search.best_score_))
print("Best estimator:\n{}".format(svm_rand_search.best_estimator_))

Best parameters: {'C': 0.05462277217684343}
Best cross-validation score: 0.89
Best estimator:
SVC(C=0.05462277217684343, class_weight='balanced', random_state=0)


In [21]:
display(recall_score(ytrain, svm_rand_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, svm_rand_search.best_estimator_.predict(Xtest)))

0.9067796610169492

0.8947368421052632

### 4. Gaussian Naive Bayes 

In [22]:
clf = GaussianNB().fit(Xtrain, ytrain)

print("Training set score: {:.3f}".format(recall_score(ytrain, clf.predict(Xtrain))))
print("Test set score: {:.3f}".format(recall_score(ytest, clf.predict(Xtest))))

print('\nParameters currently in use:')
print(clf.get_params())

Training set score: 0.743
Test set score: 0.737

Parameters currently in use:
{'priors': None, 'var_smoothing': 1e-09}


In [50]:
gnb_param_grid = {'var_smoothing': np.logspace(-10,1,100)}

gnb_rand_search = RandomizedSearchCV(GaussianNB(),
                                     gnb_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
gnb_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(gnb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gnb_rand_search.best_score_))
print("Best estimator:\n{}".format(gnb_rand_search.best_estimator_))

Best parameters: {'var_smoothing': 0.46415888336127725}
Best cross-validation score: 0.87
Best estimator:
GaussianNB(var_smoothing=0.46415888336127725)


In [51]:
display(recall_score(ytrain, gnb_rand_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, gnb_rand_search.best_estimator_.predict(Xtest)))

0.8728813559322034

0.8618421052631579

In [52]:
confusion_matrix(ytest,gnb_rand_search.predict(Xtest))

array([[1017,  454],
       [  21,  131]])

In [53]:
clf = GaussianNB().fit(Xtrain_upsampled, ytrain_upsampled)

print("Training set score: {:.3f}".format(recall_score(ytrain_upsampled, clf.predict(Xtrain_upsampled))))
print("Test set score: {:.3f}".format(recall_score(ytest, clf.predict(Xtest))))

print('\nParameters currently in use:')
print(clf.get_params())

Training set score: 0.484
Test set score: 0.000

Parameters currently in use:
{'priors': None, 'var_smoothing': 1e-09}


In [54]:
gnb_param_grid = {'var_smoothing': np.logspace(-10,1,100)}

gnb_rand_search = RandomizedSearchCV(GaussianNB(),
                                     gnb_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
gnb_rand_search.fit(Xtrain_upsampled, ytrain_upsampled)

print("Best parameters: {}".format(gnb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gnb_rand_search.best_score_))
print("Best estimator:\n{}".format(gnb_rand_search.best_estimator_))

Best parameters: {'var_smoothing': 7.742636826811262e-05}
Best cross-validation score: 0.74
Best estimator:
GaussianNB(var_smoothing=7.742636826811262e-05)


In [55]:
display(recall_score(ytrain_upsampled, gnb_rand_search.best_estimator_.predict(Xtrain_upsampled)))
display(recall_score(ytest, gnb_rand_search.best_estimator_.predict(Xtest)))

0.7343431401106904

0.0

### 5. K nearest neighbour

In [30]:
knn = KNeighborsClassifier().fit(Xtrain, ytrain)

print("Training set score: {:.3f}".format(recall_score(ytrain, knn.predict(Xtrain))))
print("Test set score: {:.3f}".format(recall_score(ytest, knn.predict(Xtest))))

print('\nParameters currently in use:')
print(knn.get_params())

Training set score: 0.531
Test set score: 0.447

Parameters currently in use:
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [31]:
knn = KNeighborsClassifier().fit(Xtrain, ytrain)

print("Training set score: {:.3f}".format(recall_score(ytrain_upsampled, knn.predict(Xtrain_upsampled))))
print("Test set score: {:.3f}".format(recall_score(ytest, knn.predict(Xtest))))

print('\nParameters currently in use:')
print(knn.get_params())

Training set score: 0.925
Test set score: 0.447

Parameters currently in use:
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [38]:
knn_param_grid = {'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size': [10,30,50],
                  'n_neighbors': [3,5,7],
                  'weights': ['uniform', 'distance']
                 }

knn_rand_search = RandomizedSearchCV(KNeighborsClassifier(),
                                     knn_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
knn_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(knn_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(knn_rand_search.best_score_))
print("Best estimator:\n{}".format(knn_rand_search.best_estimator_))

Best parameters: {'weights': 'uniform', 'n_neighbors': 3, 'leaf_size': 10, 'algorithm': 'ball_tree'}
Best cross-validation score: 0.42
Best estimator:
KNeighborsClassifier(algorithm='ball_tree', leaf_size=10, n_neighbors=3)


In [39]:
display(recall_score(ytrain, knn_rand_search.best_estimator_.predict(Xtrain)))
display(recall_score(ytest, knn_rand_search.best_estimator_.predict(Xtest)))

0.6271186440677966

0.45394736842105265

-------

# Drop features